# RGBD People dataset conversion toolkit

In [1]:
import glob
import os
import numpy as np

In [2]:
pathBasic='/home/jguerry/workspace/datasets/RGBD_people/data/track_annotations/'
pathAnnot_pv='/home/jguerry/workspace/datasets/RGBD_people/data/Annotations_pv/'
allTrack=sorted(glob.glob(pathBasic+"/Track*"))

In [3]:
myBoxes={}
for track in allTrack:
    # Read in the file
    with open(track) as f:
        content = f.readlines()
    # you may also want to remove whitespace characters like `\n` at the end of each line
    content = [x.strip().split() for x in content if x[0]!='#'] 
    for elem in content:

#         if 'seq0_0963_' not in elem[0]:
#             continue
#         print track[-6:-4],elem, elem[0].split('_')
        IMAGE_NAME_ID = elem[0]
        TS = elem[1]
        X_tl_dpt = elem[2]
        Y_tl_dpt = elem[3]
        WDT_dpt = elem[4]
        HGT_dpt = elem[5]
        X_tl_rgb = elem[6]
        Y_tl_rgb = elem[7]
        WDT_rgb = elem[8]
        HGT_rgb = elem[9]
        VSB= elem[10]
        
        
        if VSB=='0':
            difficulty=2
        elif VSB=='1':
            difficulty=0
        elif VSB=='2':
            difficulty=1
        else:
            print 'aaaaaaarrrr',VSB
                
        
        x1=int(X_tl_rgb)
        y1=int(Y_tl_rgb)
        x2=int(X_tl_rgb)+int(WDT_rgb)
        y2=int(Y_tl_rgb)+int(HGT_rgb)
        track_id=int(track[-6:-4])
        
        if IMAGE_NAME_ID not in myBoxes:
            myBoxes[IMAGE_NAME_ID]=[{'classname':'person','track_id':track_id,'difficulty':difficulty,'x1':x1,'y1':y1,'x2':x2,'y2':y2}]
        else:
            myBoxes[IMAGE_NAME_ID]=myBoxes[IMAGE_NAME_ID]+[{'classname':'person','track_id':track_id,'difficulty':difficulty,'x1':x1,'y1':y1,'x2':x2,'y2':y2}]

In [8]:
for frame in myBoxes:
    filename = pathAnnot_pv+frame+".xml"
    try:
        fd=open(filename,'w')
    except:
        print 'Could not save to the specified XML file. Please check the location. Does the directory exist?'

    print >> fd, "<annotation>"
    print >> fd, "	<folder>rgb</folder>"
    print >> fd, "	<filename>"+frame+"</filename>"
    print >> fd, "	<source>"
    print >> fd, "		<database>The RGBD People database</database>"
    print >> fd, "	</source>"
    print >> fd, "	<owner>M. Luber and L. Spinello and K. O. Arras</owner>"
    print >> fd, "	<size>"
    print >> fd, "		<width>640</width>"
    print >> fd, "		<height>480</height>"
    print >> fd, "		<depth>3</depth>"
    print >> fd, "	</size>"
    print >> fd, "	<segmented>0</segmented>"


    for boxe in myBoxes[frame]:
        print >> fd, "	<object>"
        print >> fd, "		<name>"+boxe['classname']+"</name>"
        print >> fd, "		<pose>unknown</pose>"
        print >> fd, "		<truncated>-1</truncated>"
        print >> fd, "		<difficult>"+str(boxe['difficulty'])+"</difficult>"
        print >> fd, "		<bndbox>"
        print >> fd, "			<xmin>"+str(boxe['x1'])+"</xmin>"
        print >> fd, "			<ymin>"+str(boxe['y1'])+"</ymin>"
        print >> fd, "			<xmax>"+str(boxe['x2'])+"</xmax>"
        print >> fd, "			<ymax>"+str(boxe['y2'])+"</ymax>"
        print >> fd, "		</bndbox>"
        print >> fd, "	</object>"

    print >> fd, "</annotation>"
    fd.close()